Solutions for windows notebook

In [ ]:
# Load the extension
%load_ext sql

In [ ]:
# Connect to DuckDB
# NOTE: ONLY 1 NOTEBOOK CAN BE CONNECTED TO DUCKDB AT ANY TIME
%sql duckdb:///../../tpch.db

# [Exercise] 

Write a query to calculate the daily running average of totalprice of every customer. 

Time limit during live workshop: 5 min

**Hint**: Figure out the `PARTITION BY` column first, then the `ORDER BY` column and finally the `FUNCTION` to use to compute running average.


In [ ]:
%%sql
SELECT
  o_custkey,
  o_orderdate,
  o_totalprice,
  AVG(o_totalprice) -- FUNCTION 
  OVER (
    PARTITION BY
      o_custkey -- PARTITION
    ORDER BY
      o_orderdate -- ORDER BY; ASCENDING ORDER unless specified as DESC
  ) AS running_sum
FROM
  orders
WHERE
  o_custkey = 4
ORDER BY
  o_orderdate
LIMIT
  10;

# [Exercise] From the `orders` table get the 3 lowest spending customers per day 

**Time limit during live workshop: 5 min**

**Hint**
  1. Figure out the `PARTITION BY` column first, then the `ORDER BY` column and finally the `FUNCTION` to use to compute running average.

The orders table schema is shown below:

![Orders table](../../images/orders.png)


In [ ]:
%%sql
SELECT
  *
FROM
  (
    SELECT
      o_orderdate,
      o_totalprice,
      o_custkey,
      RANK() -- RANKING FUNCTION 
      OVER (
        PARTITION BY
          o_orderdate -- PARTITION BY order date
        ORDER BY
          o_totalprice -- ORDER rows withing partition by totalprice
      ) AS rnk
    FROM
      orders
  )
WHERE
  rnk <= 3
ORDER BY
  o_orderdate
LIMIT
  5;

# [Exercise] Write a SQL query using the `orders` table that calculates the following columns:
	1. o_orderdate: From orders table
	2. o_custkey: From orders table
	3. o_totalprice: From orders table
	4. totalprice_diff: The customers current day's o_totalprice - that same customers most recent previous purchase's o_totalprice

* **Time limit during live workshop: 5 min**

* **Hint**: 
    1. Start by figuring out what the `PARTITION BY` column should be, then what the `ORDER BY` column should be, and then finally the function to use. 
    2. Use the `LAG(column_name)` ranking function to identify the prior day's revenue.

* The orders table schema is shown below:

![Orders table](../../images/orders.png)


In [ ]:
%%sql
SELECT
  o_orderdate,
  o_custkey,
  o_totalprice,
  o_totalprice - lag (o_totalprice) OVER (
    PARTITION BY
      o_custkey
    ORDER BY
      o_orderdate
  ) AS totalprice_diff
FROM
  orders
ORDER BY
  o_custkey,
  o_orderdate
LIMIT
  50;

# [Exercise]

Now that we have seen how to create a window frame with ROWS, let' explore how to do this with RANGE.

1. Write a query on the orders table that has the following output:
    1. order_month, 
    2. o_custkey,
    3. total_price,
    4. three_mo_total_price_avg
    5. **consecutive_three_mo_total_price_avg**: The consecutive 3 month average of total_price for that customer. Note that this should only include months that are chronologically next to each other.
                                                                                               
**Time limit during live workshop: 10 min**
                                                                                               
**Hint**: Use `CAST(strftime(o_orderdate, '%Y-%m-01') AS DATE)` to cast order_month to date format.

**Hint**: Use the `INTERVAL` format shown above to construct the window function to compute `consecutive_three_mo_total_price_avg` column.
                                                                                           
* The orders table schema is shown below:

![Orders table](../../images/orders.png)

In [ ]:
%%sql
SELECT
  order_month,
  o_custkey,
  total_price,
  ROUND(
    AVG(total_price) OVER (
      PARTITION BY
        o_custkey
      ORDER BY
        CAST(order_month AS DATE) RANGE BETWEEN INTERVAL '1' MONTH PRECEDING
        AND INTERVAL '1' MONTH FOLLOWING
    ),
    2
  ) AS consecutive_three_mo_total_price_avg,
  ROUND(
    AVG(total_price) OVER (
      PARTITION BY
        o_custkey
      ORDER BY
        order_month ROWS BETWEEN 1 PRECEDING
        AND 1 FOLLOWING
    ),
    2
  ) AS three_mo_total_price_avg
FROM
  (
    SELECT
      strftime (o_orderdate, '%Y-%m-01') AS order_month,
      o_custkey,
      SUM(o_totalprice) AS total_price
    FROM
      orders
    GROUP BY
      1,
      2
  )
ORDER BY
  o_custkey,
  order_month
LIMIT
  50;